In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import pandas as pd
from enbios2.base.experiment import Experiment
import bw2data

from enbios2.bw2.util import report
from enbios2.models.experiment_models import ExperimentData

In [3]:
# get an overview of brighway projects and databases
report()

Project: default
['biosphere3']
Project: ecoinvent_391
['biosphere3', 'ecoinvent_391_cutoff']
Project: ecoinvent
[]


In [4]:
# select the brightway project and database (some ecoinvent database)
PROJECT_NAME = "ecoinvent_391"
DATABASE = "ecoinvent_391_cutoff"

bw2data.projects.set_current(PROJECT_NAME)
db = bw2data.Database(DATABASE)

In [5]:
wind_turbines_spain = db.search(
    "electricity production, wind, 1-3MW turbine, onshore", filter={"location": "ES"}
)[:1]
wind_turbines_spain

Excluding 319 filtered results


['electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None)]

In [6]:
solar_spain = db.search("solar", filter={"location": "ES"})[:2]
solar_spain

Excluding 465 filtered results


['electricity production, solar tower power plant, 20 MW' (kilowatt hour, ES, None),
 'electricity production, solar thermal parabolic trough, 50 MW' (kilowatt hour, ES, None)]

In [7]:
# for the experiment we need to create a list of activities (or a dict, where the keys represent the aliases)
# We need to add the codes, otherwise the brightway search will not be not uniquely identify the activities
# adding name is just for convenience
experiment_activities = []

for activity in wind_turbines_spain + solar_spain:
    experiment_activities.append(
        {
            "id": {
                "name": activity["name"],
                "code": activity["code"],
                "alias": activity["name"],
            }
        }
    )

experiment_activities

[{'id': {'name': 'electricity production, wind, >3MW turbine, onshore',
   'code': '0d48975a3766c13e68cedeb6c24f6f74',
   'alias': 'electricity production, wind, >3MW turbine, onshore'}},
 {'id': {'name': 'electricity production, solar tower power plant, 20 MW',
   'code': 'f2700b2ffcb6b32143a6f95d9cca1721',
   'alias': 'electricity production, solar tower power plant, 20 MW'}},
 {'id': {'name': 'electricity production, solar thermal parabolic trough, 50 MW',
   'code': '19040cdacdbf038e2f6ad59814f7a9ed',
   'alias': 'electricity production, solar thermal parabolic trough, 50 MW'}}]

In [9]:
# select 2 random methods and convert them into the form for enbios2
methods = [bw2data.methods.random() for _ in range(1)]
experiment_methods = [{"id": method} for method in methods]

experiment_methods

[{'id': ('EPS 2020d no LT',
   'emissions to soil no LT',
   'monetary impact value no LT')}]

In [26]:
hierarchy = {
    "wind": [wind_act["name"] for wind_act in wind_turbines_spain],
    "solar": [solar_act["name"] for solar_act in solar_spain],
}

In [27]:
from random import randint


def create_random_scenario():
    return {
        "activities": {
            act["id"]["alias"]: ["kilowatt_hour", randint(1, 10)]
            for act in experiment_activities
        }
    }


scenarios = [create_random_scenario() for _ in range(8)]

In [28]:
# let's store the raw data, because we want to modify it later
raw_data = {
    "bw_project": PROJECT_NAME,
    "activities": experiment_activities,
    "methods": experiment_methods,
    "hierarchy": hierarchy,
    "scenarios": scenarios,
}

In [29]:
# create a experiment object. This will validate the activities, their outputs, the methods and the scenarios.
exp: Experiment = Experiment(raw_data)

In [19]:
print(exp.info())

Experiment: 
Activities: 6
  electricity production, wind, >3MW turbine, onshore - electricity production, wind, >3MW turbine, onshore
  electricity production, wind, 1-3MW turbine, onshore - electricity production, wind, 1-3MW turbine, onshore
  electricity production, wind, 1-3MW turbine, offshore - electricity production, wind, 1-3MW turbine, offshore
  electricity production, wind, <1MW turbine, onshore - electricity production, wind, <1MW turbine, onshore
  electricity production, solar tower power plant, 20 MW - electricity production, solar tower power plant, 20 MW
  electricity production, solar thermal parabolic trough, 50 MW - electricity production, solar thermal parabolic trough, 50 MW
Methods: 4
 ('ReCiPe 2016 v1.03, endpoint (E) no LT', 'human health no LT', 'climate change: human health no LT')
 ('EDIP 2003', 'human toxicity', 'via air')
 ('EF v3.1 no LT', 'human toxicity: carcinogenic no LT', 'comparative toxic unit for human (CTUh) no LT')
 ('ReCiPe 2016 v1.03, endpoin

In [20]:
exp = Experiment(raw_data)

In [30]:
_ = exp.run()

In [31]:
exp.execution_time

'0:13:29'

In [ ]:
from pathlib import Path

exp.results_to_csv("s1.csv", level_names=["root", "technology", "activity"])
df = pd.read_csv("s1.csv").fillna("")
Path("s1.csv").unlink()
df

In [23]:
from enbios2.generic.files import DataPath
import pickle

pickle.dump(exp, DataPath("test_data/exp.pkl").open("wb"))

In [32]:
activities = list(exp.scenarios[0].result_tree.get_leaves())
[a._data for a in activities]

[ScenarioResultNodeData(output=('kilowatt_hour', 8.0), results={'ReCiPe 2016 v1.03, endpoint (E) no LT_human health no LT_climate change: human health no LT': 2.2459819913683697e-06, 'EDIP 2003_human toxicity_via air': 791976.7223746454, 'EF v3.1 no LT_human toxicity: carcinogenic no LT_comparative toxic unit for human (CTUh) no LT': 9.410761255249915e-10, 'ReCiPe 2016 v1.03, endpoint (H) no LT_human health no LT_ozone depletion no LT': 3.7767645697409365e-11}, bw_activity='electricity production, wind, >3MW turbine, onshore' (kilowatt hour, ES, None)),
 ScenarioResultNodeData(output=('kilowatt_hour', 3.0), results={'ReCiPe 2016 v1.03, endpoint (E) no LT_human health no LT_climate change: human health no LT': 5.091191665386904e-07, 'EDIP 2003_human toxicity_via air': 83543.30934557733, 'EF v3.1 no LT_human toxicity: carcinogenic no LT_comparative toxic unit for human (CTUh) no LT': 2.3228561898088935e-10, 'ReCiPe 2016 v1.03, endpoint (H) no LT_human health no LT_ozone depletion no LT':

In [31]:
exp.scenarios[0].rearrange_results(
    {
        "group1": [
            "electricity production, wind, >3MW turbine, onshore",
            "electricity production, wind, 1-3MW turbine, onshore",
            "electricity production, wind, 1-3MW turbine, offshore",
        ],
        "group2": [
            "electricity production, wind, <1MW turbine, onshore",
            "electricity production, solar tower power plant, 20 MW",
            "electricity production, solar thermal parabolic trough, 50 MW",
        ],
    }
)

AssertionError: data shoould be set before (node: 'electricity production, wind, >3MW turbine, onshore')